<a href="https://colab.research.google.com/github/pathu11/NMT-Training-Speech-to-Text-Convertion/blob/main/NMT_m2m100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import Dataset
from sklearn.model_selection import train_test_split
import os


In [5]:

# 1. LOAD AND SPLIT YOUR DATA
# ---------------------------------------------------------
file_path = "/content/drive/MyDrive/research/model/merged_f.txt" # Update path if needed

src_texts = []
tgt_texts = []

print("Loading data...")
with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        if "@" in line:
            parts = line.strip().split("@")
            if len(parts) == 2:
                # Part 0 is Sinhala Text, Part 1 is Gloss IDs/Text
                # Since M2M100 expects text, we treat your Gloss string as the target text
                src_texts.append(parts[0].strip())
                tgt_texts.append(parts[1].strip())

# Split: 90% Train, 10% Eval (Validation)
train_src, eval_src, train_tgt, eval_tgt = train_test_split(src_texts, tgt_texts, test_size=0.1, random_state=42)

# Create Hugging Face Dataset objects
train_dataset = Dataset.from_dict({"source": train_src, "target": train_tgt})
eval_dataset = Dataset.from_dict({"source": eval_src, "target": eval_tgt})

print(f"Training samples: {len(train_dataset)}")
print(f"Evaluation samples: {len(eval_dataset)}")

Loading data...
Training samples: 103437
Evaluation samples: 11494


In [2]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

# Set source/target languages
tokenizer.src_lang = "si"  # Sinhala
tokenizer.tgt_lang = "si"

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

In [6]:
# 3. TOKENIZATION FUNCTION
# ---------------------------------------------------------
def preprocess_function(examples):
    inputs = examples["source"]
    targets = examples["target"]

    # Tokenize inputs
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)

    # Tokenize targets (Labels)
    # Context manager ensures we use the target language settings
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply tokenization to datasets
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_eval = eval_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/103437 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/11494 [00:00<?, ? examples/s]

In [1]:
!pip install -q transformers datasets peft accelerate evaluate scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.5 MB/s eta 0:00:00
